# Building `dms-view` datasets for [Sourisseau *et al.*, 2019](https://research.fhcrc.org/content/dam/stripe/bloom/labfiles/publications/Sourisseau2019.pdf)

This jupyter notebook builds a `dms-view` datafile for the Deep Mutational Scanning (DMS; `Sourisseau2019_DMS.csv`) and the Mutational Antigenic Profiling (MAP; `Sourisseau2019_MAP.csv`) of Zika Envelope protein.
The data is scraped from the [paper repo](https://github.com/jbloomlab/ZIKV_DMS_with_EvansLab). 

## notebook setup

In [1]:
import pandas as pd
from scipy.stats import entropy
from Bio import SeqIO
from phydmslib.constants import *
import numpy as np

## create new protein structure

In [2]:
target_chains = ['C', 'E']
with open(f'5ire_{"_".join(target_chains)}.pdb', 'w') as outfile:
    with open('5ire.pdb', 'r') as f:
        for line in f.readlines():
            if line.startswith('ATOM') and line.split()[4] not in target_chains:
                pass
            else:
                outfile.write(line)

## Deep Mutational Scanning data

The DMS data will show two "conditions": raw preferences, and rescaled preferences. 
For each "condition", the dot plot will show either the entropy, n effective, or the RSA and the logoplot will show the prefs value, the mutational effect (shared between conditions), and the natural frequencies (shared between conditions).

### paths to datafiles on github

In [3]:
muteffects_fname = ('https://raw.githubusercontent.com/jbloomlab/ZIKV_DMS_with_EvansLab/master/results/'
                    'muteffects/unscaled_muteffects.csv')
prefs_fname = ('https://raw.githubusercontent.com/jbloomlab/ZIKV_DMS_with_EvansLab/master/results/'
               'prefs/summary_avgprefs.csv')
rescaledprefs_fname = ('https://raw.githubusercontent.com/jbloomlab/ZIKV_DMS_with_EvansLab/master/results/'
                       'prefs/rescaled_prefs.csv')
sitesummary_fname= ('https://raw.githubusercontent.com/jbloomlab/ZIKV_DMS_with_EvansLab/master/results/'
                    'struct_props/struct_props_mut_tol.csv')

### RSA

In [4]:
# read in RSA and entropy/n effective 
RSA = (pd.read_csv(sitesummary_fname).query("pdb == '5ire'")
       [['site', 'RSA', 'mutational_tolerance_measure', 'mutational_tolerance']]
       .rename(columns={'RSA': 'site_RSA'})
       .drop(columns=['mutational_tolerance_measure', 'mutational_tolerance']))
RSA.head()

,site,site_RSA
0,1,0.030457
4,2,0.244526
8,3,0.023952
12,4,0.142132
16,5,0.625000


### mutational effects

In [5]:
mut_effects = (pd.read_csv(muteffects_fname)
               .drop(columns=['mutation', 'effect'])
               .rename(columns={'mutant': 'mutation', 
                                'log2effect': 'mut_mutational effect'}))
mut_effects.head()

,site,wildtype,mutation,mut_mutational effect
0,1,I,A,-5.046030
1,1,I,C,-4.308696
2,1,I,D,-3.884530
3,1,I,E,-4.290569
4,1,I,F,-4.370049


### natural frequencies

In [6]:
translate ={}
for codon in CODON_TO_INDEX.items():
    translate[codon[0]] = INDEX_TO_AA[CODON_TO_AA[codon[1]]]

In [7]:
# read in data
with open('E_alignment.fasta', "r") as handle:
    alignment = [(x.id, str(x.seq).strip()) for x in SeqIO.parse(handle, "fasta")]

seqlength = len(alignment[0][1]) // 3
nat = {k: [0 for x in range(seqlength)] for k in list(INDEX_TO_AA.values())}

# count amino acid frequencies
for seq in alignment:
    for i in range(seqlength):
        codon = seq[1][3 * i: 3 * i + 3]
        if codon != '---':
            nat[translate[codon]][i] += 1
nat = pd.DataFrame(nat)

# Normalize the dataframe
assert not np.any(nat.sum(axis=1) == 0), ("Attempting to normalize a "
                                            "site by an amino acid count"
                                            " of zero. Does the alignment"
                                            " have an all gap column?")
nat = nat.div(nat.sum(axis=1), axis=0)
assert np.allclose(nat.sum(axis=1), 1, atol=0.005)
nat["site"] = [x+1 for x in range(len(nat))]

# convert to dms-view format
nat = pd.melt(nat, id_vars='site', var_name='mutation', value_name='mut_natural frequencies')
nat.head()

,site,mutation,mut_natural frequencies
0,1,A,0.0
1,2,A,0.0
2,3,A,0.0
3,4,A,0.0
4,5,A,0.0


### preferences

In [8]:
amino_acids = ['A', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'K', 'L','M', 'N', 'P', 'Q', 'R', 'S', 'T', 'V', 'W', 'Y']
beta = 1.65
df = (pd.concat([(pd.read_csv(fname)[['site'] + amino_acids])
                    .assign(condition=condition)
                    for condition, fname in [('raw preferences', prefs_fname),
                                             ('rescaled preferences', rescaledprefs_fname)]
                   ]))
df['site_entropy'] = df[amino_acids].apply(lambda x: entropy(x), axis=1)
df['site_n effective'] = df['site_entropy'].apply(lambda x: 2**x)
df = pd.melt(df, 
                id_vars=['site', 'condition', 'site_entropy', 'site_n effective'], 
                var_name='mutation', 
                value_name='mut_pref')
df = pd.merge(df, RSA, on=['site'])
df = pd.merge(df, mut_effects, on=['site', 'mutation'])
df = pd.merge(df, nat, on=['site', 'mutation'])

# rescale mutational effects
df['mut_mutational effect'] = (df[['mut_mutational effect', 'condition']]
                               .apply(lambda x: 
                                      x[0] * beta if x[1] == 'rescaled preferences' else x[0], axis=1))
df.head()

,site,condition,site_entropy,site_n effective,mutation,mut_pref,site_RSA,wildtype,mut_mutational effect,mut_natural frequencies
0,1,raw preferences,1.809948,3.506295,A,0.005438,0.030457,I,-5.046030,0.0
1,1,raw preferences,1.809948,3.506295,A,0.005438,0.030457,I,-5.046030,0.0
2,1,rescaled preferences,1.317192,2.491807,A,0.000505,0.030457,I,-8.325950,0.0
3,1,rescaled preferences,1.317192,2.491807,A,0.000505,0.030457,I,-8.325950,0.0
4,1,raw preferences,1.809948,3.506295,C,0.009065,0.030457,I,-4.308696,0.0


In [9]:
df['protein_chain'] = f"{' '.join(target_chains)}"
df['protein_site'] = df['site']
df['label_site'] = df[['wildtype', 'site']].apply(lambda x: f'{x[0]} {x[1]}', axis=1)
df = df.drop_duplicates()
df.head()

,site,condition,site_entropy,site_n effective,mutation,mut_pref,site_RSA,wildtype,mut_mutational effect,mut_natural frequencies,protein_chain,protein_site,label_site
0,1,raw preferences,1.809948,3.506295,A,0.005438,0.030457,I,-5.046030,0.0,C E,1,I 1
2,1,rescaled preferences,1.317192,2.491807,A,0.000505,0.030457,I,-8.325950,0.0,C E,1,I 1
4,1,raw preferences,1.809948,3.506295,C,0.009065,0.030457,I,-4.308696,0.0,C E,1,I 1
6,1,rescaled preferences,1.317192,2.491807,C,0.001174,0.030457,I,-7.109348,0.0,C E,1,I 1
8,1,raw preferences,1.809948,3.506295,D,0.012163,0.030457,I,-3.884530,0.0,C E,1,I 1


In [10]:
df.to_csv('Sourisseau2019_DMS.csv', index=False)

## Mutational Antigenic Profiling Data

The MAP data will show two "conditions": antibody ZKA185 and antibody ZKA64. 

### differential selection

In [2]:
conditions = ['ZKA64', 'ZKA185']

In [55]:
df = []

# process data from the two antibodies
for condition in conditions:
    # mutation differential selection
    mut = (pd.read_csv(('https://raw.githubusercontent.com/jbloomlab/ZIKV_DMS_with_EvansLab/master/results/'
                       f'diffsel/summary_{condition}-meanmutdiffsel.csv'))
           .rename(columns={'mutdiffsel': 'mut_diffsel'}))

    # site differential selection
    site = pd.read_csv(('https://raw.githubusercontent.com/jbloomlab/ZIKV_DMS_with_EvansLab/master/results/'
                       f'diffsel/summary_{condition}-meansitediffsel.csv'))
    site.columns = [f"site_{' '.join(x.split('_'))}" if x!='site' else x for x in site.columns.values]

    # all together
    tog = pd.merge(mut, site, on='site')
    tog['condition'] = condition
    df.append(tog)
df = pd.concat(df)

# add in protein information and site information
df['label_site'] = df[['wildtype', 'site']].apply(lambda x: f'{x[0]} {x[1]}', axis=1)
df['protein_chain'] = 'A'
df['protein_site'] = df['site']

# calculate positive differential selection
df['mut_positive diffsel'] = mut['mut_diffsel'].clip(lower=0)

# reorder the columns
firstmut = 'mut_positive diffsel'
firstsite = 'site_positive diffsel'
cols = list(df.columns.values) 
cols.remove(firstmut) 
cols.remove(firstsite) 
df = df[[firstmut, firstsite] + cols]
df.head()

,mut_positive diffsel,site_positive diffsel,site,wildtype,mutation,mut_diffsel,site_abs diffsel,site_negative diffsel,site_max diffsel,site_min diffsel,condition,label_site,protein_chain,protein_site
0,7.664267,40.671283,333,A,T,6.442699,41.707742,-1.036459,6.442699,-0.542144,ZKA64,A 333,A,333
1,7.384974,40.671283,333,A,E,5.739223,41.707742,-1.036459,6.442699,-0.542144,ZKA64,A 333,A,333
2,7.262515,40.671283,333,A,R,4.704770,41.707742,-1.036459,6.442699,-0.542144,ZKA64,A 333,A,333
3,7.177435,40.671283,333,A,V,4.428613,41.707742,-1.036459,6.442699,-0.542144,ZKA64,A 333,A,333
4,5.764480,40.671283,333,A,L,4.418945,41.707742,-1.036459,6.442699,-0.542144,ZKA64,A 333,A,333


In [ ]:
df.to_csv('Sourisseau2019_MAP.csv', index=Fal)